In [91]:
import tempfile
import numpy as np
import pandas as pd
import collections
import re
import os
import subprocess
import pybedtools
from intervaltree import Interval, IntervalTree
from pybedtools import BedTool
from Bio import SeqIO

In [92]:
# simulation parameters
seed = 123
N_VARS = 10
N_BACKGROUND_GENES = 100
N_EXONS = 2 # fuse first N exons to last N exons
MIN_EXONS = 3
BLOCK_RANGE = (30, 200)
FOLD = 50
FRAG_SIZE = 300
FRAG_SD = 20
READ_LEN = 150

In [93]:
# references
ART_ILLUMINA = '/Users/marek.cmero/apps/art_bin_MountRainier/ART_ILLUMINA'
GENOME_FASTA = '/Users/marek.cmero/reference/fastas/Homo_sapiens.GRCh38.dna.primary_assembly.fa'
GTF_REF = '/Users/marek.cmero/reference/gtf/chess_mini_chr12_ref.gtf'
FASTA_OUT = 'cryptic_variants_simu.fasta'
CONTROL_FASTA_OUT = 'cryptic_variants_control.fasta'
OUTDIR = '/Users/marek.cmero/Desktop/output'

# cleanup (from older runs)
if os.path.exists(FASTA_OUT):
    os.remove(FASTA_OUT)
if os.path.exists(CONTROL_FASTA_OUT):
    os.remove(CONTROL_FASTA_OUT)

In [94]:
# functions
def get_gene_name(row):
    '''
    Prevents KeyError if gene name missing
    '''
    try:
        return row.attrs['gene_name']
    except KeyError:
        return ''
    
def get_seq(gr):
    '''
    Returns a dictionary of exon sequences
    and the corresponding strand of the transcript
    '''
    block_seqs = gr.sequence(fi=GENOME_FASTA, s=True)
    block_dict = collections.OrderedDict()
    with tempfile.NamedTemporaryFile() as fa_tmp:
        fa_tmp.write(bytes(open(block_seqs.seqfn).read(), 'utf-8'))
        fa_tmp.flush()

        for record in SeqIO.parse(fa_tmp.name, 'fasta'):
            block_dict[record.id] = str(record.seq)

    strand = re.search('\(([-+])\)', next(iter(block_dict.keys())))
    assert strand
    
    return block_dict, strand.group(1)

def write_sequence(seq_dict, strand, output_file, name):
    '''
    Writes sequence dictionary to output file
    '''
    seq = [seq_dict[ex] for ex in seq_dict.keys()]
    seq = seq if strand == '+' else [s for s in reversed(seq)]

    with open(output_file, 'a') as fout:
        fout.write('>%s\n' % name)
        fout.write(''.join(seq) + '\n')
        
def get_chrom_features(chrom, gr):
    '''
    Get all merged intervals on given chromosome
    '''
    chrom_features = gr.filter(lambda x: x.chrom == chrom).merge()    
    chrom_features = [(g.start, g.end) for g in gr]
    
    chrom_tree = IntervalTree()
    [chrom_tree.addi(s, e) for s, e in chrom_features]

    return chrom_tree

def get_gene_features(chroms, gr):
    '''
    Get interval tree for start/ends for 
    each gene on each chromosome
    '''
    gn_ref = pd.DataFrame([(g.chrom, g.start, g.end, get_gene_name(g)) for g in gr])
    aggregator = {1: lambda x: min(x),
                  2: lambda x: max(x)}
    gn_ref = gn_ref.groupby([0, 3], as_index=False, sort=False).agg(aggregator)
    gn_ref = gn_ref[[0, 1, 2, 3]]
    gn_ref.columns = ['chrom', 'start', 'end', 'gene']
    gn_ref = gn_ref[gn_ref.gene!='']

    ref_trees = {}
    for chrom in chroms:
        chr_ref = gn_ref[gn_ref.chrom == chrom]
        ref_tree = IntervalTree()
        for s,e,g in zip(chr_ref['start'].values, chr_ref['end'].values, chr_ref['gene'].values):
            ref_tree.addi(s-1, e, g)
    ref_trees[chrom] = ref_tree
    
    return ref_trees

def get_ee_seq(ex_list, gr):
    '''
    Extends given exon by random size and returns
    its sequence. If a reference exon exists that
    already extends the given exon, then extend
    past any other reference exons.
    '''
    # TODO: make sure exon size doesn't extend to the next exon 
    block_size = np.random.randint(BLOCK_RANGE[0], BLOCK_RANGE[1])
    block = ex_list[1] if s1 == '+' else ex_list[0]
    loc = re.compile('[:\-\(\)]').split(block)
    start = loc[2] if s1 == '+' else str(int(loc[1]) - block_size)
    end = str(int(loc[2]) + block_size) if s1 == '+' else loc[1]

    ex = get_chrom_features(loc[0], gr.merge())
    olap = ex.overlap(int(start), int(end))
    if len(olap) > 0:        
        end = str(int(list(olap)[0][1]) + block_size) if s1 == '+' else end
        start = str(int(list(olap)[0][0]) - block_size) if s1 == '-' else start
    
    block_bed = '%s\t%s\t%s\t.\t1\t%s' % (loc[0], start, end, s1)
    block_bt = BedTool(block_bed, from_string=True)
    block_seq, bs = get_seq(block_bt)
    ext_seq = ''.join([bs for bs in block_seq.values()])
    
    return ext_seq

def get_random_block(chroms, gene_trees, seed):
    '''
    Get random block sequence for feature
    not overlapping any other genomic features
    '''    
    block_size = np.random.randint(BLOCK_RANGE[0], BLOCK_RANGE[1])
    chrom = np.random.choice(chroms)
    chrom_features = gene_trees[chrom]
    
    chr_range = chr_sizes[('chr%s' % chrom)]
    block_start = np.random.randint(chr_range[0], chr_range[1]-block_size)
    block_end = block_start + block_size        
    
    seq = 'N'
    while 'N' in seq:
        # only select sequence if there's no Ns
        while chrom_features.overlaps(block_start, block_end):
            seed += 123
            np.random.seed(seed)
            block_start = np.random.randint(chr_range[0], chr_range[1]-block_size)
            block_end = block_start + block_size

        strand = np.random.choice(['+','-'])
        block_bed = '%s\t%d\t%d\t.\t1\t%s' % (chrom, block_start, block_end, strand)
        block_bt = BedTool(block_bed, from_string=True)
        block_seq, bs = get_seq(block_bt)
        seq = ''.join([bs for bs in block_seq.values()])

    return block_seq, seed

def write_fusion(tx1, tx2, all_exons, name, add=['none', 'EE']):
    '''
    Get left and right sequences of given transcripts
    corresponding to the first N exons and last N exons of
    transcripts 1 and 2 respectively.
    Automatically writes wild type transcript to CONTROL_FASTA_OUT.
    '''
    exons1 = all_exons.filter(lambda x: x['transcript_id'] == tx1).saveas()
    exons2 = all_exons.filter(lambda x: x['transcript_id'] == tx2).saveas()

    tx1_seq, s1 = get_seq(exons1)
    tx2_seq, s2 = get_seq(exons2)

    # write wild-type transcripts
    write_sequence(tx1_seq, s1, CONTROL_FASTA_OUT, tx1)
    write_sequence(tx2_seq, s2, CONTROL_FASTA_OUT, tx2)
        
    # pick N 5' exons for transcript 1
    ex1_list = [ex for ex in tx1_seq.keys()]
    ex1_list = ex1_list[:N_EXONS] if s1 == '+' else ex1_list[-N_EXONS:]

    ext_seq = ''
    if add == 'EE':
        ext_seq = get_ee_seq(ex1_list, all_exons)
    
    # pick N 3' exons for transcript 2
    ex2_list = [ex for ex in tx2_seq.keys()]
    ex2_list = ex2_list[-N_EXONS:] if s2 == '+' else ex2_list[:N_EXONS]

    # select sequences and reverse order if antisense
    seq1 = [tx1_seq[ex] for ex in ex1_list]
    seq1 = seq1 if s1 == '+' else [s for s in reversed(seq1)]   

    seq2 = [tx2_seq[ex] for ex in ex2_list]
    seq2 = seq2 if s2 == '+' else [s for s in reversed(seq2)]
    
    seq = ''.join(seq1 + [ext_seq] + seq2)
    
    # write output
    with open(FASTA_OUT, 'a') as fout:
        fout.write('>%s\n' % name)
        fout.write(seq + '\n')

In [95]:
%%time
# build GTF reference
gr = BedTool(GTF_REF) 

# ensure each transcript in reference has at least N exons
all_exons = gr.filter(lambda x: x[2] == 'exon').saveas()
all_txs = [(tx['transcript_id'], get_gene_name(tx)) for tx in all_exons]
valid_txs = pd.DataFrame(pd.Series(all_txs).value_counts(), columns=['exon_count'])
valid_txs = valid_txs[valid_txs.exon_count >= MIN_EXONS]
valid_txs = valid_txs.index.values

all_genes = np.unique([gene for tx, gene in valid_txs if gene != ''])
var_genes = np.empty(0)

chr_sizes = pybedtools.chromsizes('hg38')
chroms = np.unique([x.chrom for x in all_exons])

# make gene start/end reference
gene_trees = get_gene_features(chroms, gr)

CPU times: user 11.4 s, sys: 230 ms, total: 11.6 s
Wall time: 11.9 s


## Generate canonical fusions

Select `N_VARS` random genes with `N_VARS` random partners and fuse first `N_EXONS` exons to `N_EXONS` terminal exons.

In [96]:
%%time
# pick fusion genes
np.random.seed(seed)
fus_genes = np.random.choice(all_genes, N_VARS * 2, replace=False)
fusions = zip(fus_genes[:N_VARS], fus_genes[N_VARS:])
fus_txs = [] # which fusions were used in the fusion; for reference

# make fusion genes
for gene1, gene2 in fusions:
    print('Generating %s:%s fusion...' % (gene1, gene2))

    # select first transcript from each gene
    tx1 = [tx for tx, gn in valid_txs if gn == gene1][0]
    tx2 = [tx for tx, gn in valid_txs if gn == gene2][0]
    fus_txs.append((tx1, tx2))

    name = '%s:%s' % (gene1, gene2)
    seq = write_fusion(tx1, tx2, all_exons, name)

Generating NCKAP1L:ALKBH2 fusion...
Generating LOC105369676:FAM216A fusion...
Generating DIABLO:LOC105370063 fusion...
Generating NOP2:SLC6A13 fusion...
Generating MRPS35:LOC105369758 fusion...
Generating WNT10B:ANKRD33 fusion...
Generating ESPL1:DHH fusion...
Generating RNFT2:LINC01559 fusion...
Generating MYF5:YARS2 fusion...
Generating ARL6IP4:NRIP2 fusion...
CPU times: user 49.4 s, sys: 507 ms, total: 49.9 s
Wall time: 51.2 s


## Generate unpartnered fusions

In [97]:
%%time
np.random.seed(seed)

var_genes = fus_genes.copy()
available_genes = list(set(all_genes).symmetric_difference(var_genes))
ufus_genes = np.random.choice(available_genes, N_VARS, replace=False)
var_genes = np.concatenate([var_genes, ufus_genes])

for idx, gene in enumerate(ufus_genes):
    # select first transcript from each gene
    tx = [tx for tx, gn in valid_txs if gn == gene][0]
        
    # get sequence
    exons = all_exons.filter(lambda x: x['transcript_id'] == tx).saveas()
    tx_seq, s = get_seq(exons)

    # write wild-type transcript
    write_sequence(tx_seq, s, CONTROL_FASTA_OUT, tx)
        
    # pick N 5' exons for transcript
    ex_list = [ex for ex in tx_seq.keys()]
    ex_list = ex_list[:N_EXONS] if s == '+' else ex_list[-N_EXONS:]
    
    # select sequences and reverse order if antisense
    seq1 = [tx_seq[ex] for ex in ex_list]
    seq1 = seq1 if s == '+' else [s for s in reversed(seq1)]
    
    block_seq, seed = get_random_block(chroms, gene_trees, seed)
    seq2 = [s for s in block_seq.values()]
    
    seq = ''.join(seq1 + seq2)
    bloc = ''.join([k for k in block_seq.keys()])
    fus_txs.append((tx, bloc))
    
    print('Generating %s:%s unpartnered fusion...' % (gene, bloc)) 
    # write output
    with open(FASTA_OUT, 'a') as fout:
        fout.write('>%s:%s\n' % (gene, bloc))
        fout.write(seq + '\n')


Generating PIANP:12:65977522-65977661(+) unpartnered fusion...
Generating TUBA1A:12:28596154-28596187(-) unpartnered fusion...
Generating LOC101927484:12:118813329-118813521(+) unpartnered fusion...
Generating ZFC3H1:12:83194082-83194240(+) unpartnered fusion...
Generating HPD:12:88615334-88615413(+) unpartnered fusion...
Generating LOC105369877:12:65690383-65690574(-) unpartnered fusion...
Generating IL22:12:65135549-65135734(+) unpartnered fusion...
Generating EEA1:12:52503345-52503375(+) unpartnered fusion...
Generating LOC101927882:12:55220154-55220207(+) unpartnered fusion...
Generating PLXNC1:12:43569701-43569897(-) unpartnered fusion...
CPU times: user 25.4 s, sys: 360 ms, total: 25.7 s
Wall time: 26.9 s


## Fusions with extended exon at boundary

In [98]:
%%time
# pick fusion genes
np.random.seed(seed)
available_genes = list(set(all_genes).symmetric_difference(var_genes))
efus_genes = np.random.choice(available_genes, N_VARS*2, replace=False)
var_genes = np.concatenate([var_genes, efus_genes])

# make fusion genes
fusions = zip(efus_genes[:N_VARS], efus_genes[N_VARS:])
for gene1, gene2 in fusions:
    print('Generating %s:%s fusion...' % (gene1, gene2))
    
    # select first transcript from each gene
    tx1 = [tx for tx, gn in valid_txs if gn == gene1][0]
    tx2 = [tx for tx, gn in valid_txs if gn == gene2][0]
    fus_txs.append((tx1, tx2))

    name = '%s:%s' % (gene1, gene2)
    seq = write_fusion(tx1, tx2, all_exons, name, add='EE')

Generating LOC105369826:PFDN5 fusion...
Generating LOC105369896:SDR9C7 fusion...
Generating C12orf76:LOC101927058 fusion...
Generating LOC105370008:LOC100506159 fusion...
Generating USP5:LOC105370080 fusion...
Generating IKZF4:LOC107984468 fusion...
Generating LOC105369875:RASSF9 fusion...
Generating LOC105370010:ZC3H10 fusion...
Generating FGF6:ZNF641 fusion...
Generating SCN8A:MED21 fusion...
CPU times: user 56.9 s, sys: 718 ms, total: 57.6 s
Wall time: 1min 1s


In [99]:
# transcript IDs used for cryptic variants
fus_txs

[('CHS.11800.1', 'CHS.12716.1'),
 ('CHS.11139.1', 'CHS.12756.5'),
 ('CHS.13049.14', 'CHS.13194.5'),
 ('CHS.10783.2', 'CHS.10619.8'),
 ('CHS.11278.2', 'CHS.11546.3'),
 ('CHS.11540.2', 'CHS.11645.2'),
 ('CHS.11726.4', 'CHS.11551.3'),
 ('CHS.12898.3', 'CHS.11083.1'),
 ('CHS.12272.1', 'CHS.11365.5'),
 ('CHS.13078.3', 'CHS.10681.8'),
 ('CHS.10796.9', '12:65977522-65977661(+)'),
 ('CHS.11556.2', '12:28596154-28596187(-)'),
 ('CHS.11813.3', '12:118813329-118813521(+)'),
 ('CHS.12168.2', '12:83194082-83194240(+)'),
 ('CHS.13034.8', '12:88615334-88615413(+)'),
 ('CHS.12312.1', '12:65690383-65690574(-)'),
 ('CHS.12105.1', '12:65135549-65135734(+)'),
 ('CHS.12410.4', '12:52503345-52503375(+)'),
 ('CHS.10844.3', '12:55220154-55220207(+)'),
 ('CHS.12439.8', '12:43569701-43569897(-)'),
 ('CHS.12146.1', 'CHS.11727.10'),
 ('CHS.12373.1', 'CHS.11912.6'),
 ('CHS.12743.3', 'CHS.11421.2'),
 ('CHS.12880.1', 'CHS.10949.2'),
 ('CHS.10812.8', 'CHS.13263.6'),
 ('CHS.11860.6', 'CHS.11876.3'),
 ('CHS.12300.1', '

In [100]:
%%time
# write background genes
np.random.seed(seed)
available_genes = list(set(all_genes).symmetric_difference(var_genes))
bg_set = np.random.choice(available_genes, N_BACKGROUND_GENES)
for gene in bg_set:
    tx = [tx for tx, gn in valid_txs if gn == gene][0]
    exons = all_exons.filter(lambda x: x['transcript_id'] == tx).saveas()
    tx_seq, strand = get_seq(exons)
    write_sequence(tx_seq, strand, CONTROL_FASTA_OUT, tx)
    write_sequence(tx_seq, strand, FASTA_OUT, tx)

CPU times: user 3min 57s, sys: 2.04 s, total: 3min 59s
Wall time: 4min 5s


In [101]:
%%time
# generate reads with art illumina
np.random.seed(seed)
seeds = np.random.randint(0, 99999, 2)

subprocess.call(['mkdir', '-p', OUTDIR])

# generate case sample
subprocess.call([ART_ILLUMINA, '-ss', 'HS25', '-i', FASTA_OUT, 
                 '-p', '-l', str(READ_LEN), '-f', str(FOLD), '-m', str(FRAG_SIZE),
                 '-s', str(FRAG_SD), '-rs', str(seeds[0]), '-o', '%s/case_R' % OUTDIR])

# generate control
subprocess.call([ART_ILLUMINA, '-ss', 'HS25', '-i', CONTROL_FASTA_OUT, 
             '-p', '-l', str(READ_LEN), '-f', str(FOLD), '-m', str(FRAG_SIZE),
             '-s', str(FRAG_SD), '-rs', str(seeds[1]), '-o', '%s/control_R' % OUTDIR])

CPU times: user 3.38 ms, sys: 14.8 ms, total: 18.2 ms
Wall time: 9.83 s


In [102]:
%%time
for sample in ['case', 'control']:
    for r in range(2):
        outf = open('%s/%s_R%d.fastq.gz' % (OUTDIR, sample, (r+1)), 'w')
        subprocess.call(['gzip', '-c', '%s/%s_R%d.fq' % (OUTDIR, sample, (r+1))], stdout=outf)
        outf.close()

CPU times: user 3.85 ms, sys: 19.9 ms, total: 23.8 ms
Wall time: 11 s
